In [23]:
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os

In [24]:
load_dotenv()

driver_conn = "{"+os.getenv('DRIVER')+"}"
server_conn = os.getenv('DB_SERVER')
database = os.getenv('DATABASE')
uid = os.getenv('UID')
password = os.getenv('PASSWORD')

cnxn_str = (f"Driver={driver_conn};"
            f"Server={server_conn};"
            f"Database={database};"
            f"UID={uid};"
            f"PWD={password};")
cnxn = pyodbc.connect(cnxn_str)

cursor = cnxn.cursor()

query = "select top(3) * from Regsosek22.dbo.tab_view_k where [r407] <= 4 AND [r427] IS NULL"

# Open and read the file as a single buffer
fd = open('query/Pengeluaran Miskin 0.sql', 'r')
sqlFile = fd.read()
fd.close()

data = pd.read_sql(sqlFile, cnxn)

data

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19004\3863271100.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sqlFile, cnxn)


,kode_kab,kode_kec,kode_desa,kode_sls,kode_subsls,sls_nama,no urut bangunan,no urut keluarga setelah verif,Status Keluarga,pengeluaran,operator,kode pml,nama pml,kode ppl,nama ppl
0,06,010,010,0007,00,RT 007 DUSUN MAKMUR,23,24,1,None,TOMO,0101,ABDUS SAMAD,01013,RINI NUR LESTARI
1,06,010,010,0015,00,RT 015 DUSUN KENANGAN,44,69,1,None,SUTRISNO,0101,ABDUS SAMAD,01015,Camadi D
2,06,010,010,0015,00,RT 015 DUSUN KENANGAN,35,101,1,None,SUTRISNO,0101,ABDUS SAMAD,01015,Camadi D
3,06,010,011,0010,00,RT 001 DUSUN DURI IV,53,94,2,None,RIDWAN,0102,ABDUL KADIR,01023,DWI EKO HARDYANSYAH
4,06,010,012,0002,00,RT 09 RW 01,31,38,2,None,JESSICA,0202,ARDI RAHMAT HIDAYATULLAH,02021,Bambang suardi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,06,060,008,0017,00,RT 017 RW 05,3,3,1,None,SUHAILI,1605,Anis mayana,16053,HENDRI
201,06,060,009,0001,00,RT 001 DUSUN BERINGIN,015,21,2,None,KHAIRUL,1606,Afrindo sabari,16063,HASNAWATI
202,06,060,009,0007,00,RT 007 DUSUN TUMENGGUNG,13,17,1,None,SUHAILI,1606,Afrindo sabari,16062,Rosmiyati
203,06,060,009,0007,00,RT 007 DUSUN TUMENGGUNG,28,33,1,None,SUHAILI,1606,Afrindo sabari,16062,Rosmiyati


In [73]:
import connector.database as dbConnector
import extract as e
import transform as t
import load as l
# library untuk mereload library yang kita buat. 
# tanpa ini, perubahan di self-made library bakal ngga kerekam 
import importlib
importlib.reload(dbConnector)
importlib.reload(e)
importlib.reload(t)
importlib.reload(l)

<module 'load' from 'c:\\Users\\ASUS\\Desktop\\Project\\etl-pipeline-bps\\load.py'>

In [79]:

sqlServerConnection = dbConnector.SQLServerConnection(driver=os.getenv('DRIVER'),
                                                      server=os.getenv('DB_SERVER'),
                                                      database = os.getenv('DATABASE'),
                                                      user_id = os.getenv('UID'),
                                                      password = os.getenv('PASSWORD'))
sqlServerConnection.connect()

# Extract from sql server db
extractor = e.Extract()
query = "select top(3) * from Regsosek22.dbo.tab_view_k where [r407] <= 4 AND [r427] IS NULL"
queryResult, columns = extractor.FromDb(sqlServerConnection, query)

# Transform to dataframe
transformer = t.Transform()
transformedData = transformer.queryResultToDF(queryResult, columns)
transformedData.head()

# Load to csv
loader = l.Load()
loader.ToFSCSV(transformedData, 'hasil/test.csv')



success load to csv
